In [0]:
from pyspark.sql.functions import col, when,row_number, max, min ,to_date, lit, when, month, datediff, year, regexp_replace, date_format, unix_timestamp, from_unixtime, current_date, count, struct, greatest,sum, avg, upper

In [0]:
spark.conf.set("fs.azure.account.key.vvprdstgdatalake.dfs.core.windows.net", dbutils.secrets.get(scope = "vvprdstgdatalake", key = "account_key1"))

In [0]:
df_motor = (spark.table("app_atendimento.tb_motor_sap")
            .select("unidade_negocio", "cpf_cnpj", "pedido", "entrega", "idcredito", "forma_restituicao","valor_do_credito", "direcionamento",                                 "faixa_aging", "area_finalizadora", "status_finalizador", "data_criacao_credito", "forma_pagamento", "sequencial_credito",                                 "sequencial_forma")   
            .withColumn("data_criacao_credito", unix_timestamp("data_criacao_credito", "dd/MM/yyyy"))
            .withColumn("data_criacao_credito", from_unixtime("data_criacao_credito", "yyyy-MM-dd"))
            .withColumn("year", year("data_criacao_credito")).filter((col("year") >= 2020))
            .filter(col("forma_restituicao").isin("DEPOSITO EM CONTA","ORDEM PAGAMENTO","ESTORNO DE CARTAO")))     
#display(df_motor)

In [0]:
### Pegando dados do motor, filtrando pelo campo forma_restituicao = VALE
df_motor_credito_vale = (spark.table("app_atendimento.tb_motor_sap")
                        .select("unidade_negocio", "cpf_cnpj", "pedido", "entrega", "idcredito", "forma_restituicao","valor_do_credito", "direcionamento",                                 "faixa_aging", "area_finalizadora", "status_finalizador", "data_criacao_credito", "forma_pagamento",                                                       "sequencial_credito", "sequencial_forma")   
                        .withColumn("data_criacao_credito", unix_timestamp("data_criacao_credito", "dd/MM/yyyy"))
                        .withColumn("data_criacao_credito", from_unixtime("data_criacao_credito", "yyyy-MM-dd")) 
                        .withColumn("year", year("data_criacao_credito")).filter((col("year") >= 2020))
                        .filter(col("forma_restituicao").isin("VALE")))    

### Query para obter do FRONT, as entregas VALE
vale = spark.sql(""" 
SELECT  Context_Site.Compra.IdUnidadeNegocio,
        Context_Site.Compra.IdCompra,
        Context_Site.CompraCliente.CpfCnpj,
        Context_Site.CompraEntrega.GerencialId,    
        (SELECT MIN (CESL.Data)
         FROM Context_Site.CompraEntregaStatusLog CESL
         WHERE CESL.IdCompraEntrega = Context_Site.CompraEntrega.IdCompraEntrega
               AND CESL.Bandeira = Context_Site.CompraEntrega.Bandeira
               AND CESL.IdCompraEntregaStatus IN ('VAL', 'RDV', 'DVC')),            
        Context_Site.FreteEntregaTipo.Nome
    
FROM   Context_Site.Compra                        
INNER  JOIN Context_Site.CompraEntrega              ON  Context_Site.CompraEntrega.IdCompra                   = Context_Site.Compra.IdCompra
                                                    AND Context_Site.CompraEntrega.Bandeira                   = Context_Site.Compra.Bandeira
INNER  JOIN Context_Site.CompraEntregaEstorno       ON  Context_Site.CompraEntregaEstorno.IdCompraEntregaNova = Context_Site.CompraEntrega.IdCompraEntrega
INNER  JOIN Context_Site.UnidadeNegocio             ON  Context_Site.UnidadeNegocio.IdUnidadeNegocio          = Context_Site.Compra.IdUnidadeNegocio      
LEFT   JOIN Context_Site.CompraEntregaStatusEstorno ON  Context_Site.CompraEntregaStatusEstorno.IdCompra      = Context_Site.Compra.IdCompra
                                                    AND Context_Site.CompraEntregaStatusEstorno.Bandeira      = Context_Site.Compra.Bandeira
INNER  JOIN Context_Site.CompraCliente              ON  Context_Site.CompraCliente.IdCompra                   = Context_Site.Compra.IdCompra
                                                    AND Context_Site.CompraCliente.Bandeira                   = Context_Site.Compra.Bandeira
INNER  JOIN Context_Site.FreteEntregaTipo           ON  Context_Site.FreteEntregaTipo.IdFreteEntregaTipo = Context_Site.CompraEntrega.IdFreteEntregaTipo
WHERE  Context_Site.CompraEntregaEstorno.TipoPagamento = 2    
       AND Context_Site.CompraCliente.TipoCliente = 0  
       AND Context_Site.CompraEntrega.Origem = 'TD' 
""")

### Join do Dataframe filtrado apenas VALE do motor com o Dataframe extraido da Query do FRONT
df_vale = (df_motor_credito_vale.join(vale, df_motor_credito_vale.entrega == vale.GerencialId)
                                .drop("IdUnidadeNegocio","IdCompra","CpfCnpj","GerencialId","scalarsubquery(IdCompraEntrega)","Nome")
                                .distinct())

#display(df_vale)

In [0]:
df_motor_credito = df_motor.unionByName(df_vale)

In [0]:
### Carregando tabelas - Loja ###
df_htr = (spark.table("raw_loja.htr_crd_fma_pgt").select("cd_ccdvl","cd_ccfpgt", "cd_ccfpgt_seq", "cd_fmapgt",                                                                                                      "ts_hcfpgt","st_ccfpgt_ret_sap")
                                                 .withColumn("ts_hcfpgt_format", to_date(col("ts_hcfpgt")))
                                                 .withColumn("data_status_aberto", 
                                                              when(col("ST_CCFPGT_RET_SAP") == 0, col("ts_hcfpgt_format")))
                                                 .withColumn("dt_status_confirmado", 
                                                              when(col("ST_CCFPGT_RET_SAP") == 1, col("ts_hcfpgt_format")))
                                                 .withColumn("dt_status_rejeitado", 
                                                              when(col("ST_CCFPGT_RET_SAP") == 2, col("ts_hcfpgt_format")))
                                                 .withColumn("dt_erro_no_processamento", 
                                                              when(col("ST_CCFPGT_RET_SAP") == 4, col("ts_hcfpgt_format")))
                                                 .withColumn("dt_status_processado", 
                                                              when(col("ST_CCFPGT_RET_SAP") == 5, col("ts_hcfpgt_format"))))
                                                                                                
df_htr_crd_fma_pgt = (df_htr.groupBy(col("cd_ccdvl"), col("cd_ccfpgt"), col("cd_ccfpgt_seq"), col("cd_fmapgt"))
                                    .agg(min(col("data_status_aberto")).alias("data_status_aberto"),
                                         min(col("dt_status_confirmado")).alias("dt_status_confirmado"),
                                         min(col("dt_status_rejeitado")).alias("dt_status_rejeitado"),
                                         min(col("dt_erro_no_processamento")).alias("dt_erro_no_processamento"),
                                         min(col("dt_status_processado")).alias("dt_status_processado")))  


df_crd_can_fma_pgt = spark.table("context_loja.crd_can_fma_pgt").select("cd_ccdvl", col("dt_ccfpgt_lan_doc").alias("data_que_saiu_motor"))
df_etn_crt_crd_scp = spark.table("raw_loja.etn_crt_crd_scp_streaming").select("cd_ccdvl", col("ts_eccscp").alias("data_inclusao_scope"))

df_loja = (df_crd_can_fma_pgt.join(df_htr_crd_fma_pgt, "cd_ccdvl", "left")  
                             .join(df_etn_crt_crd_scp, "cd_ccdvl", "left").distinct())

#display(df_loja)

In [0]:
df_motor_loja = df_motor_credito.join(df_loja,(df_loja.cd_ccdvl == df_motor_credito.idcredito) &
                                              (df_loja.cd_fmapgt == df_motor_credito.forma_pagamento) &
                                              (df_loja.cd_ccfpgt_seq == df_motor_credito.sequencial_credito) &
                                              (df_loja.cd_ccfpgt == df_motor_credito.sequencial_forma), "left")

In [0]:
### Carregando tabelas Front
df_compra_entrega = spark.table("context_site.compraentrega").select("idfreteentregatipo", "idcompraentrega", col("gerencialid").alias("entrega"),
                                                                     "idlojista", "bandeira") 
df_frete_entrega_tipo = spark.table("context_site.freteentregatipo").select(col("nome").alias("tipo_entrega"), "idfreteentregatipo")
df_lojista = (spark.table("context_site.lojista").withColumn("mktplc" , when((col("idlojista").isin(15,16,10037)) | 
                                                                             col("retiroemloja") == True,"Não").otherwise("Sim"))   
                                                 .select("idlojista","mktplc", col("bandeira").alias("bandeira")))


### Montando Dataframe Front              
df_front = (df_compra_entrega.join(df_frete_entrega_tipo, "idfreteentregatipo")
                             .join(df_lojista, (df_compra_entrega.idlojista == df_lojista.idlojista) &
                                               (df_compra_entrega.bandeira == df_lojista.bandeira))
                             .select("entrega", "tipo_entrega", "mktplc"))

In [0]:
df = df_motor_loja.join(df_front, "entrega").distinct()

In [0]:
# criando colunas q serao usadas no relatorio
base_restituicao = (df.withColumn("mes_elegivel", 
                                   when(to_date("data_que_saiu_motor").isNull(), "Ainda Não Elegivel").otherwise(month(to_date("data_que_saiu_motor"))))
                      .withColumn("ano_elegivel", 
                                   when(to_date("data_que_saiu_motor").isNull(), "Ainda Não Elegivel").otherwise(year(to_date("data_que_saiu_motor"))))
                      .withColumn("origem_pedido", when(col("pedido") < 167000000, "LN/SIGE").otherwise("Legado"))
                      .withColumn("lead_ate_elegivel", 
                                   when((col("data_criacao_credito").isNull()) | (col("data_que_saiu_motor").isNull()), "Ainda Nao Elegivel")
                                  .when((col("data_criacao_credito").isNotNull()) & (col("data_que_saiu_motor").isNotNull()), 
                                         datediff("data_que_saiu_motor","data_criacao_credito")))
                      .withColumn("lead_confirmado",
                                   when(col("lead_ate_elegivel").isNull(), "Em Processo")
                                  .when((col("forma_restituicao") != "ORDEM PAGAMENTO") & (col("dt_status_confirmado").isNull()), "Não Confirmado") 
                                  .when((col("forma_restituicao") == "ORDEM PAGAMENTO") & (col("dt_status_confirmado").isNull()), "Não Confirmado")
                                  .when((col("forma_restituicao") == "ORDEM PAGAMENTO") & (col("dt_status_processado").isNotNull()), 
                                         datediff("dt_status_processado","data_que_saiu_motor"))
                                  .when(col("forma_restituicao") != "ORDEM PAGAMENTO", datediff("dt_status_confirmado","data_que_saiu_motor")))
                      .withColumn("lead_total",
                                   when(col("lead_confirmado").isNull(), "Em Processo")
                                  .when((col("forma_restituicao") != "ORDEM PAGAMENTO") & (col("dt_status_confirmado").isNull()), "Em Processo")
                                  .when((col("forma_restituicao") == "ORDEM PAGAMENTO") & (col("dt_status_confirmado").isNull()) &
                                        (col("dt_status_processado").isNull()), "Em Processo")
                                  .when(col("forma_restituicao") != "ORDEM PAGAMENTO", datediff("dt_status_confirmado","data_criacao_credito"))
                                  .otherwise(datediff("dt_status_processado","data_criacao_credito")))  
                      .withColumn("processo_nao_elegivel",
                                   when((col("forma_restituicao") == "ORDEM PAGAMENTO") & (col("dt_status_confirmado").isNull()) & 
                                        (col("dt_status_processado").isNull()), "Em Processo")
                                  .when((col("forma_restituicao") != "ORDEM PAGAMENTO") & (col("dt_status_confirmado").isNull()), 
                                         datediff(current_date(),"data_criacao_credito"))
                                  .otherwise(datediff(current_date(), "data_criacao_credito")))
                      .withColumn("processo_elegivel",
                                   when((col("forma_restituicao") == "ORDEM PAGAMENTO") & (col("dt_status_confirmado").isNull()) &
                                        (col("dt_status_processado").isNull()), "Em Processo")
                                  .when((col("forma_restituicao") != "ORDEM PAGAMENTO") & (col("dt_status_confirmado").isNull()),
                                         datediff(current_date(), "data_criacao_credito"))
                                  .otherwise(datediff(current_date(), "data_que_saiu_motor")))
                      .withColumn("dt_status_confirmado",
                                   when(col("forma_restituicao") == "VALE", col("data_que_saiu_motor"))
                                  .otherwise(col("dt_status_confirmado")))
                    
                                  .drop("cd_ccdvl", "cd_ccfpgt", "cd_ccfpgt_seq", "cd_fmapgt")
                                  .select("unidade_negocio","cpf_cnpj","pedido","entrega","idcredito","forma_restituicao","valor_do_credito",
                                          "direcionamento","faixa_aging","area_finalizadora","status_finalizador","data_criacao_credito",
                                          "data_que_saiu_motor","data_inclusao_scope","data_status_aberto","dt_status_confirmado",
                                          "dt_status_rejeitado","dt_erro_no_processamento","dt_status_processado","tipo_entrega","mktplc",
                                          "mes_elegivel","ano_elegivel","origem_pedido","lead_ate_elegivel","lead_confirmado","lead_total",
                                          "processo_nao_elegivel","processo_elegivel"))

base_restituicao.cache       

In [0]:
a = base_restituicao.limit(30000)

In [0]:
display(a.filter(col("forma_restituicao") == 'VALE'))

In [0]:
display(a.filter((a.data_criacao_credito >= '2020-01-01') & (a.data_criacao_credito <= '2020-03-31')))

In [0]:
# amostra = base_restituicao.limit(1000)
# amostra.repartition(1).write.format("csv").option("sep", ";").option("header","true").mode("overwrite").save("/mnt/gen2/dev/Leonardo_Andrade/Amostra")

In [0]:
display(a.filter((a.data_criacao_credito >= '2020-01-01') & (a.data_criacao_credito <= '2020-03-31')))